In [1]:
# coding=utf-8
from collections import defaultdict

import numpy as np

In [2]:
def generate_sequences(n):
    result = []
    formatter = ('{0:0' + str(n) + 'b}')
    for i in range(2 ** n):
        result.append([[int(j)] for j in formatter.format(i)])
    return np.array(result)

In [3]:
class F2Matrix(object):
    def __init__(self, matrix):
        self.matrix = matrix
        self.height = len(matrix)
        self.len = len(matrix[0])

    def swap_columns(self, i, j):
        for row in self.matrix:
            row[i], row[j] = row[j], row[i]

    def swap_rows(self, i, j):
        self.matrix[i], self.matrix[j] = self.matrix[j], self.matrix[i]

    def subtract_row(self, from_ind, which_ind):
        zipped = zip(self.matrix[from_ind], self.matrix[which_ind])
        self.matrix[from_ind] = map(lambda x: (x[0] + x[1]) % 2, zipped)
        
    def subtract_col(self, from_ind, which_ind):
        for row in self.matrix:
            row[from_ind] = (row[from_ind] + row[which_ind]) % 2

    def transpose(self):
        return F2Matrix(zip(*self.matrix))

    def __str__(self):
        return '\n'.join([str(row) for row in self.matrix])


In [4]:
f2_matrix = F2Matrix([
        [1, 0, 0, 0, 0, 0, 0, 1, 0, 1],
        [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
        [0, 0, 1, 0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 1, 0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 1, 0, 0, 1, 1]]
    )

In [5]:
f2_matrix.subtract_row(0, 5)
f2_matrix.subtract_row(2, 5)
f2_matrix.subtract_row(4, 5)
f2_matrix.subtract_row(0, 4)
f2_matrix.subtract_row(2, 4)
f2_matrix.subtract_row(3, 4)
f2_matrix.subtract_row(0, 2)
f2_matrix.subtract_row(1, 2)
f2_matrix.subtract_row(0, 1)
print f2_matrix

[1, 1, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 1, 1, 0, 1, 0, 0, 0, 1, 0]
[0, 0, 1, 0, 1, 0, 1, 0, 0, 0]
[0, 0, 0, 1, 1, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 1, 1]


In [6]:
f2_matrix.subtract_col(8, 5)
print f2_matrix.matrix

[[1, 1, 0, 0, 1, 0, 0, 0, 0, 0], [0, 1, 1, 0, 1, 0, 0, 0, 1, 0], [0, 0, 1, 0, 1, 0, 1, 0, 0, 0], [0, 0, 0, 1, 1, 1, 0, 0, 1, 0], [0, 0, 0, 0, 1, 1, 1, 1, 1, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 1]]


In [7]:
f2_matrix.subtract_row(1, 4)
f2_matrix.subtract_row(3, 4)
print f2_matrix

[1, 1, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 1, 1, 0, 0, 1, 1, 1, 0, 0]
[0, 0, 1, 0, 1, 0, 1, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 0, 1, 1, 1, 1, 1, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1]


In [8]:
f2_matrix.subtract_row(1, 3)
print f2_matrix

[1, 1, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 0, 1, 0, 0, 0, 0]
[0, 0, 1, 0, 1, 0, 1, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 0, 1, 1, 1, 1, 1, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1]


In [9]:
def get_spans(g):
    spans = []
    for row in g:
        size = len(row)
        min_ind, max_ind, ind = size, 0, 0
        while ind < size:
            if row[ind] == 1:
                max_ind = ind - 1
                if min_ind == size:
                    min_ind = ind
            ind += 1
        spans.append((min_ind, max_ind))
    return spans

spans = get_spans(f2_matrix.matrix)
print(spans)

[(0, 3), (1, 4), (2, 5), (3, 6), (4, 7), (5, 8)]


In [10]:
level_symbols = defaultdict(lambda: [])
for ind, span in enumerate(spans):
    for j in range(span[0], span[1] + 1):
        level_symbols[j].append(ind)
print(level_symbols)

defaultdict(<function <lambda> at 0x7f181cb3d5f0>, {0: [0], 1: [0, 1], 2: [0, 1, 2], 3: [0, 1, 2, 3], 4: [1, 2, 3, 4], 5: [2, 3, 4, 5], 6: [3, 4, 5], 7: [4, 5], 8: [5]})


In [11]:
class Vertex(object):
    def __init__(self, code):
        self.code = code
        self.edges = {}

    def add_edge(self, val, to):
        self.edges[to] = val

    def __str__(self):
        return ''.join([str(j) for j in self.code])
    
def transpose_sequence(seq):
    return [l[0] for l in seq.tolist()]

In [12]:
def has_edge(prev_v, v, prev_level, level):
    for code, level in [(prev_v, prev_level), (v, level)]:
        if len(code) != len(level):
            return True
    ind_value = {}
    for ind, i in enumerate(prev_level):
        ind_value[i] = prev_v[ind]
    for ind, i in enumerate(level):
        if (i in ind_value.keys()) and (ind_value[i] != v[ind]):
            return False
    return True

In [13]:
def count_sgn(prev_v, v, prev_level, level, col):
    ind_value = {}
    for ind, i in enumerate(prev_level):
        ind_value[i] = prev_v[ind]
    for ind, i in enumerate(level):
        ind_value[i] = v[ind]
    sum = 0
    for k, v in ind_value.iteritems():
        sum += v * col[k]
    return (sum % 2)

In [14]:
matrix = np.array(f2_matrix.matrix)
vertices = defaultdict(lambda: [])
vertices[-1] = [Vertex([0])]
levels = matrix.shape[1]
for i in range(levels):
    for sequence in generate_sequences(len(level_symbols[i])):
        v = Vertex(transpose_sequence(sequence))
        vertices[i].append(v)
        col = matrix[:, i]
        for prev_v in vertices[i - 1]:
            if has_edge(prev_v.code, v.code, level_symbols[i - 1], level_symbols[i]):  
                prev_v.add_edge(count_sgn(prev_v.code, v.code, level_symbols[i - 1], level_symbols[i], col), v)
for i in range(levels):
    print('-'.join([str(v) for v in vertices[i]]))

0-1
00-01-10-11
000-001-010-011-100-101-110-111
0000-0001-0010-0011-0100-0101-0110-0111-1000-1001-1010-1011-1100-1101-1110-1111
0000-0001-0010-0011-0100-0101-0110-0111-1000-1001-1010-1011-1100-1101-1110-1111
0000-0001-0010-0011-0100-0101-0110-0111-1000-1001-1010-1011-1100-1101-1110-1111
000-001-010-011-100-101-110-111
00-01-10-11
0-1
0


# Перевод в DOT

In [15]:
def dot_vertex(level, v):
    return 'V_{}_{}'.format(level + 1, str(v))


def graph_to_dot(levels, vertices):
    print('digraph item_set {')
    for i in range(-1, levels):
        print('//level {}'.format(i))
        for v in vertices[i]:
            print('{}[label = \"{}\"];'.format(dot_vertex(i, v), str(v)))
            for nxt, val in v.edges.iteritems():
                print('{} -> {} [label = \"{}\"];'.format(dot_vertex(i, v), dot_vertex(i + 1, nxt), str(val)))
    print('}')
    
graph_to_dot(levels, vertices)

digraph item_set {
//level -1
V_0_0[label = "0"];
V_0_0 -> V_1_0 [label = "0"];
V_0_0 -> V_1_1 [label = "1"];
//level 0
V_1_0[label = "0"];
V_1_0 -> V_2_00 [label = "0"];
V_1_0 -> V_2_01 [label = "1"];
V_1_1[label = "1"];
V_1_1 -> V_2_10 [label = "1"];
V_1_1 -> V_2_11 [label = "0"];
//level 1
V_2_00[label = "00"];
V_2_00 -> V_3_000 [label = "0"];
V_2_00 -> V_3_001 [label = "1"];
V_2_01[label = "01"];
V_2_01 -> V_3_010 [label = "1"];
V_2_01 -> V_3_011 [label = "0"];
V_2_10[label = "10"];
V_2_10 -> V_3_101 [label = "1"];
V_2_10 -> V_3_100 [label = "0"];
V_2_11[label = "11"];
V_2_11 -> V_3_111 [label = "0"];
V_2_11 -> V_3_110 [label = "1"];
//level 2
V_3_000[label = "000"];
V_3_000 -> V_4_0001 [label = "1"];
V_3_000 -> V_4_0000 [label = "0"];
V_3_001[label = "001"];
V_3_001 -> V_4_0011 [label = "1"];
V_3_001 -> V_4_0010 [label = "0"];
V_3_010[label = "010"];
V_3_010 -> V_4_0101 [label = "0"];
V_3_010 -> V_4_0100 [label = "1"];
V_3_011[label = "011"];
V_3_011 -> V_4_0111 [label = "0"];
V_3

# Проверочная матрица

In [16]:
matrix = np.array([
    [1, 1, 0, 0, 0, 0, 0, 0, 1, 1],
    [0, 1, 1, 0, 1, 1, 0, 0, 1, 0],
    [1, 0, 0, 0, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 1, 0, 1, 0, 1, 0, 1]
])

In [25]:
def multiply(msg, matrix):
    return np.remainder(np.dot(msg, matrix), 2)

def to_str(v):
    return ''.join([str(i) for i in v])

def get_slice(matrix, i):
    return matrix[:, :(i + 1)]

vertices = defaultdict(lambda: {})
vertices[-1] = {'0000':Vertex(['0000'])}
for i in range(10):
    for sequence in generate_sequences(i + 1):
        syn = multiply(get_slice(matrix, i), sequence)
        code = to_str(transpose_sequence(syn))
        vertices[i][code] = Vertex(code)
    for k, v in vertices[i - 1].iteritems():
        prev_syn = [int(c) for c in k]
        new_code = []
        for j in range(4):
            new_code.append((prev_syn[j] + matrix[j][i]) % 2)
        v.add_edge(0, vertices[i][k])
        v.add_edge(1, vertices[i][to_str(new_code)])

In [26]:
def delete_bad_vertices(vertices):
    for i in reversed(range(10)):
        for k, v in vertices[i].items():
            if (i == 9) and (k != '0000'):
                vertices[i].pop(k)
            elif (i != 9):
                has_edge = False
                for next_v, val in v.edges.items():
                    if next_v.code in vertices[i + 1].keys():
                        has_edge = True
                    else:
                        v.edges.pop(next_v)
                if not has_edge:
                    vertices[i].pop(k)

delete_bad_vertices(vertices)                    

for i in range(-1, 10):
    print('-'.join([k for k, v in vertices[i].iteritems()]))

0000
1010-0000
1010-0110-0000-1100
0110-0000-0010-0100-1110-1100-1010-1000
0110-0111-0000-0001-0011-0010-0101-0100-1111-1110-1100-1101-1010-1011-1001-1000
0110-0111-0000-0001-0011-0010-0101-0100-1111-1110-1100-1101-1010-1011-1001-1000
0110-0111-0000-0001-0011-0010-0101-0100-1111-1110-1100-1101-1010-1011-1001-1000
0111-0000-0011-0100-1110-1101-1010-1001
0111-0000-1110-1001
0000-1001
0000


In [27]:
print('digraph item_set {')
for i in range(-1, 10):
    print('//level {}'.format(i))
    for k, v in vertices[i].iteritems():
        print('{}[label = \"{}\"];'.format(dot_vertex(i, v), str(v)))
        for nxt, val in v.edges.iteritems():
            print('{} -> {} [label = \"{}\"];'.format(dot_vertex(i, v), dot_vertex(i + 1, nxt), str(val)))
print('}')

digraph item_set {
//level -1
V_0_0000[label = "0000"];
V_0_0000 -> V_1_0000 [label = "0"];
V_0_0000 -> V_1_1010 [label = "1"];
//level 0
V_1_1010[label = "1010"];
V_1_1010 -> V_2_1010 [label = "0"];
V_1_1010 -> V_2_0110 [label = "1"];
V_1_0000[label = "0000"];
V_1_0000 -> V_2_0000 [label = "0"];
V_1_0000 -> V_2_1100 [label = "1"];
//level 1
V_2_1010[label = "1010"];
V_2_1010 -> V_3_1010 [label = "0"];
V_2_1010 -> V_3_1110 [label = "1"];
V_2_0110[label = "0110"];
V_2_0110 -> V_3_0110 [label = "0"];
V_2_0110 -> V_3_0010 [label = "1"];
V_2_0000[label = "0000"];
V_2_0000 -> V_3_0000 [label = "0"];
V_2_0000 -> V_3_0100 [label = "1"];
V_2_1100[label = "1100"];
V_2_1100 -> V_3_1100 [label = "0"];
V_2_1100 -> V_3_1000 [label = "1"];
//level 2
V_3_0110[label = "0110"];
V_3_0110 -> V_4_0111 [label = "1"];
V_3_0110 -> V_4_0110 [label = "0"];
V_3_0000[label = "0000"];
V_3_0000 -> V_4_0001 [label = "1"];
V_3_0000 -> V_4_0000 [label = "0"];
V_3_0010[label = "0010"];
V_3_0010 -> V_4_0011 [label = "1